## Agent 생성 및 실행

In [ ]:
pip install agent-framework --pre

In [ ]:
pip install --upgrade pywin32

#### .env 파일 로드

In [ ]:
from dotenv import load_dotenv
load_dotenv()  # .env 파일 로드

#### Agent Framework API 확인

설치된 agent-framework의 실제 클래스와 메서드를 확인합니다.

In [ ]:
# agent_framework 패키지의 실제 구조 확인
import agent_framework
from agent_framework.azure import AzureOpenAIChatClient

print("=== agent_framework 모듈 확인 ===")
print(f"agent_framework 버전: {agent_framework.__version__ if hasattr(agent_framework, '__version__') else 'N/A'}")
print(f"\nagent_framework 모듈 내용:")
print([x for x in dir(agent_framework) if not x.startswith('_')])

print("\n=== AzureOpenAIChatClient 클래스 확인 ===")
print("사용 가능한 메서드:")
methods = [m for m in dir(AzureOpenAIChatClient) if not m.startswith('_')]
for method in methods:
    print(f"  - {method}")

print("\n=== AzureOpenAIChatClient 초기화 파라미터 확인 ===")
import inspect
sig = inspect.signature(AzureOpenAIChatClient.__init__)
print(f"파라미터: {sig}")


#### 에이전트 생성

먼저 Azure OpenAI와 통신하기 위한 채팅 클라이언트를 생성하고, 필수 조건 단계에서 Azure CLI를 사용하여 인증할 때 사용했던 것과 동일한 로그인 정보를 사용하십시오. 다음으로, 에이전트를 생성하고 에이전트에 대한 지침과 이름을 입력합니다.

In [ ]:
import asyncio
from agent_framework.azure import AzureOpenAIChatClient
from azure.identity import AzureCliCredential

agent = AzureOpenAIChatClient(credential=AzureCliCredential()).create_agent(
    instructions="You are good at telling jokes.",
    name="Joker"
)

## 참조 : https://github.com/warnov/ms-agent-framework-step-by-step-workshop
## endpoint와 deployment_name을 .env에 설정된 기본값이 아닌 환경 변수에서 읽도록 수정하려면 다음과 같이 변경
## 기본값 Key는 AZURE_OPENAI_ENDPOINT와 AZURE_OPENAI_CHAT_DEPLOYMENT_NAME
# agent = AzureOpenAIChatClient(
#    credential=AzureCliCredential(),
#    endpoint=os.environ["AOAI_ENDPOINT"],
#    deployment_name=os.environ["AOAI_DEPLOYMENT"]
#).create_agent(
#    instructions="You are good at telling tales.",
#    name="Joker"
#)

#### 에이전트 실행

에이전트를 실행하려면 에이전트 인스턴스에서 run 메서드를 호출하고 사용자 입력을 제공하십시오. 에이전트는 응답 객체를 반환하며, result.text 속성에 접근하면 에이전트가 반환한 텍스트 결과를 확인할 수 있습니다.

In [ ]:
async def main():
    result = await agent.run("Tell me a joke about a pirate.")
    print(result.text)

await main()

#### 스트리밍을 사용하여 에이전트 실행

스트리밍 방식으로 에이전트를 실행하려면 에이전트 인스턴스의 run_stream 메서드를 호출하고 사용자 입력을 제공하십시오. 에이전트는 update 객체 목록을 스트리밍 방식으로 전송하며, 각 업데이트 객체의 text 속성에 접근하면 해당 업데이트에 포함된 텍스트 결과의 일부를 얻을 수 있습니다.

In [ ]:
async def main():
    async for update in agent.run_stream("Tell me a long joke about a pirate."):
        if update.text:
            print(update.text, end="", flush=True)
    print()  # New line after streaming is complete

await main()

#### ChatMessage를 사용하여 에이전트 실행 (이미지 포함)

단순 문자열 대신 하나 이상의 ChatMessage 객체를 run 및 run_stream 메서드에 제공할 수도 있습니다. 예제는 온라인 이미지를 사용하여 ChatMessage 객체를 생성하는 방법을 보여줍니다.

In [10]:
from agent_framework import ChatMessage, TextContent, UriContent, Role

message = ChatMessage(
    role=Role.USER,
    contents=[
        TextContent(text="Tell me a joke about this image?"),
        UriContent(uri="https://www.fotosanimales.es/wp-content/uploads/2017/12/pinguino.jpg", media_type="image/jpeg")
    ]
)

async def main():
    result = await agent.run(message)
    print(result.text)

await main()

Why don’t penguins ever get invited to dinner parties?

Because they always bring ice to the conversation and just "chill" in the corner! 🐧😄


DataContent를 사용하여 로컬 파일 시스템에서 이미지를 불러오는 방법도 있습니다

In [ ]:
from agent_framework import ChatMessage, TextContent, DataContent, Role

# Load image from local file
with open("image/image1.jpg", "rb") as f:
    image_bytes = f.read()

message = ChatMessage(
    role=Role.USER,
    contents=[
        TextContent(text="What do you see in this image?"),
        DataContent(
            data=image_bytes,
            media_type="image/jpeg"
        )
    ]
)

async def main():
    result = await agent.run(message)
    print(result.text)

await main()